# Experimentation and uplift testing

In [1]:
# Load required packages
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv('QVI_data.csv')
data

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream
...,...,...,...,...,...,...,...,...,...,...,...,...
264829,2370701,2018-12-08,88,240378,24,Grain Waves Sweet Chilli 210g,2,7.2,210,GRNWVES,YOUNG FAMILIES,Mainstream
264830,2370751,2018-10-01,88,240394,60,Kettle Tortilla ChpsFeta&Garlic 150g,2,9.2,150,KETTLE,YOUNG FAMILIES,Premium
264831,2370961,2018-10-24,88,240480,70,Tyrrells Crisps Lightly Salted 165g,2,8.4,165,TYRRELLS,OLDER FAMILIES,Budget
264832,2370961,2018-10-27,88,240481,65,Old El Paso Salsa Dip Chnky Tom Ht300g,2,10.2,300,OLD,OLDER FAMILIES,Budget


In [7]:
data['MONTH_ID'] = pd.to_datetime(data['DATE']).dt.strftime('%Y%m')
data['MONTH_ID'] = data['MONTH_ID'].astype(int)
data['MONTH_ID']

0         201810
1         201809
2         201903
3         201903
4         201811
           ...  
264829    201812
264830    201810
264831    201810
264832    201810
264833    201812
Name: MONTH_ID, Length: 264834, dtype: int32

In [10]:
metrics = data.groupby(['MONTH_ID'])[['TOT_SALES', 'LYLTY_CARD_NBR', 'TXN_ID', 'PROD_QTY']].agg({
    'TOT_SALES': 'sum', 'LYLTY_CARD_NBR' : 'nunique', 'TXN_ID' : 'nunique', 'PROD_QTY' : 'sum'
})

metrics['TXN_PER_CUST'] = metrics['TXN_ID'] / metrics['LYLTY_CARD_NBR']
metrics['QTY_PER_CUST'] = metrics['PROD_QTY'] / metrics['LYLTY_CARD_NBR']
metrics['AVG_PRICE'] = metrics['TOT_SALES'] / metrics['PROD_QTY']
metrics

,TOT_SALES,LYLTY_CARD_NBR,TXN_ID,PROD_QTY,TXN_PER_CUST,QTY_PER_CUST,AVG_PRICE
MONTH_ID,,,,,,,
201807,165275.30,18780,22435,43242,1.194622,2.302556,3.822101
201808,158081.05,18718,22264,41284,1.189443,2.205578,3.829112
201809,160522.00,18191,21593,41792,1.187016,2.297400,3.840974
201810,164415.70,18570,22157,42821,1.193161,2.305924,3.839604
201811,160233.70,18299,21710,41895,1.186404,2.289469,3.824650
201812,167913.40,19076,22683,43845,1.189086,2.298438,3.829705
201901,162642.30,18463,22000,42501,1.191572,2.301955,3.826788
201902,150665.00,17249,20266,39220,1.174909,2.273755,3.841535
201903,166265.20,18897,22447,43347,1.187861,2.293856,3.835680
